In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
import data_prepare.misc as misc
import cv2
import numpy as np
import math

# name = "ev_hiy_04d.png"
name = "ev_hiy_04e.png"
img_gt = cv2.imread(f"E:/Dataset/hamidashi/lr/{name}")
img = cv2.imread(f"E:/Dataset/hamidashi/hr/{name}")
img = misc.resize(img, tuple(img_gt.shape[:2][::-1]), interpolation = misc.Filter.LANCZOS)

def f(amount, threshold, radius = 3):
        out = misc.ringing(img, radius, amount, threshold)
        return math.log(1 - misc.ssim(out, img_gt))
    
# Generate x and y values
tick = 11
x_vals = np.linspace(0.0, 0.5, tick)
y_vals = np.linspace(0, 1.5, tick)
radius = 1.5
x_vals = np.linspace(0.26, 0.3, tick)
y_vals = np.linspace(0.7, 0.9, tick)

# Create an empty grid to store function values
z = np.zeros((len(x_vals), len(y_vals)))

# Define a function to evaluate f(x, y) in parallel
def evaluate_point(i, j):
    return i, j, f(x_vals[i], y_vals[j], radius)

# Evaluate the function at each point on the grid in parallel
with ThreadPoolExecutor() as executor:
    futures = []
    for i in range(len(x_vals)):
        for j in range(len(y_vals)):
            futures.append(executor.submit(evaluate_point, i, j))
    
    for future in futures:
        result = future.result()
        z[result[1], result[0]] = result[2]

# Create a 2D contour plot
X, Y = np.meshgrid(x_vals, y_vals)
plt.contourf(X, Y, z, 50, cmap='plasma')
plt.colorbar(label='f(x, y)')

# Set labels and title
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Contour Plot of f(x, y)')

# Show plot
plt.show()
print(np.min(z))


In [ ]:
################### Test single image ###########################
import cv2 
import data_prepare.misc as misc
import math

lr_img = cv2.imread("R:/hamidashi/lr/ev_asu_02a.png")
hr_img = cv2.imread("R:/hamidashi/hr/ev_asu_02a.png")
chainner_img = cv2.imread("R:/ev_asu_02a.png")
size = tuple(lr_img.shape[:2][::-1])
hr_img = misc.resize(hr_img, size, interpolation = misc.Filter.LANCZOS)
print(math.log10(1 - misc.ssim(hr_img, lr_img)))
def usm(a,b,c):
    print(math.log10(1 - misc.ssim(misc.ringing(hr_img, a, b, c), lr_img)))

usm(3, 0.08, 0.25)
usm(3, 0.085, 1)
usm(3, 0.135, 0.5)
usm(3, 0.0, 0.0)

In [45]:
################### Test multi image ###########################
# bigfolder = "E:/Dataset/otome/_0_/"
# smalfolder = "E:/Dataset/otome/_2_/"
bigfolder = "R:/hamidashi/hr/"
smalfolder = "R:/hamidashi/lr/"
property = [1.5, 0.29, 0.72] # -5.589691205463635
property = [1.5, 0.28, 0.78] # -5.592861432717648
property = [1.5, 0.28, 0.75] # -5.5929928869166154
import os
import cv2 
import data_prepare.misc as misc
import numpy as np
import math

def process_file(name):
    lr_img = cv2.imread(f"{smalfolder}{name}")
    hr_img = cv2.imread(f"{bigfolder}{name}")
    hr_img = misc.resize(hr_img, tuple(lr_img.shape[:2][::-1]), interpolation=misc.Filter.LANCZOS)
    x1 = math.log(1 - misc.ssim(hr_img, lr_img))
    hr_img = misc.ringing(hr_img, *property) # -5.5929928869166154
    x2 = math.log(1 - misc.ssim(hr_img, lr_img))
    return name, x1, x2

ori = []
usm = []
files = []

for root, dir, file_names in os.walk(bigfolder):
    files.extend(file_names)

with ThreadPoolExecutor() as executor:
    results = executor.map(process_file, files)

for name, x1, x2 in results:
    ori.append(x1)
    usm.append(x2)
    if x1-x2 < 0.4 or x2>-5:
        print(name, x1, x2)

print(f"{np.average(ori) = }")
print(f"{np.average(usm) = }")

np.average(ori) = 0.0
np.average(usm) = -5.592861432717648
